# Chapter2. NLP 기술 빠르게 훑어보기
- 자연어처리(NLP) : 정보 추출, 자동 음성 인식, 기계번역, 감성 분석, 질의 응답, 요약 같은 실용적인 문제를 해결하는 방법을 개발하는 것
- 전산 언어학(Computational Linguistics) : 컴퓨터 언어의 특징을 이해하는 방법을 개발

### 1. 말뭉치, 토큰, 타입
- 모든 NLP 작업은 말뭉치 corpus라 부르는 텍스트 데이터로 시작
- 말뭉치는 일반적으로 원시 텍스트와 이 텍스트에 연관된 메타데이터를 포함
- 원시 텍스트는 일반적으로 문자를 토큰token 이라는 연속된 단어로 묶었을 때 유용
- 영어에서 토큰은 공백문자나 구두점으로 구분되는 단어와 숫자에 해당
- 머신러닝 분야에서 메터데이터가 붙은 텍스트를 샘플sample 또는 데이터 포인트 data point라고 부르며 샘플의 모음인 말뭉치는 데이터셋dataset 이라고 한다
- 토큰화tokenization는 텍스트를 토큰으로 나누는 과정. 오픈소스 NLP 패키지는 기본적인 토큰화를 제공
- 타입type은 말뭉치에 등장하는 고유한 토큰. 말뭉치에 있는 모든 타입의 집합이 어휘사전 또는 어휘lexicon
- 단어는 내용어content words와 불용어stopword로 구분. 관사와 전치사 같은 불용어는 대부분 내용어를 보충하는 문법적인 용도로 사용
- 특성공학 feature engineering : 언어학을 이해하고 NLP 문제 해결에 적용하는 과정을 의미

In [4]:
# pip install spacy
# pip install spacy && python -m spacy download en

In [11]:
# 텍스트 토큰화
import spacy
nlp = spacy.load('en_core_web_sm')
text = "Mary, don't slap the green witch"
print([str(token) for token in nlp(text.lower())])

['mary', ',', 'do', "n't", 'slap', 'the', 'green', 'witch']


In [14]:
from nltk.tokenize import TweetTokenizer
tweet = u"Snow White and the Seven Degrees #MakeAMovieCold@midnight:-_"
tokenizer= TweetTokenizer()
print(tokenizer.tokenize(tweet.lower()))

['snow', 'white', 'and', 'the', 'seven', 'degrees', '#makeamoviecold', '@midnight', ':', '-', '_']


## 2. 유니그램, 바이그램, 트라이그램, ..., n-그램
- n-gram은 텍스트에 있는 고정 길이(n)의 연속된 토큰 시퀀스

In [1]:
# 텍스트에서 n-그램 만들기
def n_grams(text, n):
    '''
    takes tokens or text, returns a list of n-grams
    '''
    return [text[i:i+n] for i in range(len(text)-n+1)]

cleaned = ["mary", ",", "n't", "slap", "green", "witch", "."]
n_grams(cleaned, 3)

[['mary', ',', "n't"],
 [',', "n't", 'slap'],
 ["n't", 'slap', 'green'],
 ['slap', 'green', 'witch'],
 ['green', 'witch', '.']]

## 3. 표제어와 어간
- 표제어lemma는 단어의 기본형
- 표제어 추출lemmatization : 토큰을 표제어로 바꾸어 벡터 표현의 차원을 줄이는 방법
- 어간 추출stemming : 표제어 추출 대신에 사용하는 축소 기법으로 수동으로 만든 규칙을 사용해 단어의 끝을 잘라 어간stem이라는 공통 형태로 축소

In [3]:
# 표제어 추출
import spacy
nlp = spacy.load('en_core_web_sm')
doc = nlp(u"he was running late")
for token in doc:
    print('{} --> {}'.format(token, token.lemma_))

he --> he
was --> be
running --> run
late --> late


In [6]:
# 어간 추출
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
stemmer = PorterStemmer()

doc = "he was running late"
token_doc = word_tokenize(doc)

for token in token_doc:
    print('{} --> {}'.format(token, stemmer.stem(token)))

he --> he
was --> wa
running --> run
late --> late


## 4. 문장과 문서 분류하기
- 문서 분류 작업은 NLP 분야의 초기 애플리케이션 중 하나
- TF와 TF-IDF표현이 문서나 문장 같은 긴 텍스트 뭉치를 분류하는데 유용
- 토픽 레이블 할당, 리뷰의 감성 예측, 스팸 이메일 필터링, 언어 식별, 이메일 분류 같은 작업은 지도 학습 기반의 문서 분류 문제

## 5. 단어 분류하기 : 품사 태깅
- 문서에 레이블을 할당하는 개념을 단어나 토큰으로 확장
- 단어 분류 작업의 예로는 품사part-of-speech(POS) 태킹tagging

In [7]:
import spacy
nlp = spacy.load('en_core_web_sm')
doc = nlp(u"Mary slapped the green witch.")
for token in doc:
    print('{} - {}'.format(token, token.pos_))

c:\Users\GIGABYTE\anaconda3\lib\site-packages\spacy\util.py:1676: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


Mary - PROPN
slapped - VERB
the - DET
green - ADJ
witch - NOUN
. - PUNCT


## 6. 청크 나누기와 개체명 인식
- 종종 연속된 여러 토큰으로 구분되는 텍스트 구에 레이블을 할당
- 예를들어 명사구(NP)와 동사구(VP)를 구별하는 것과 같은 작업을 청크 나누기chunking 또는 부분 구문 분석shallow parsing라고 한다
- 부분 구문 분석의 목적은 명사, 동사, 형용사 같은 문법 요소로 구성된 고차원의 단어를 유도해 내는 것
- 부분 구문 분석 모델 훈련에 사용할 데이터가 없다면 품사 태깅에 정규식을 적용해 부분 구문 분석을 근사
- 또 다른 유용한 단위는 개체명named entity. 개체명은 사람, 장소, 회사, 약 이름과 같은 실제 세상의 개념을 의미하는 문자열

In [9]:
# 명사구(NP) 부분 구문 분석
import spacy
nlp = spacy.load('en_core_web_sm')
doc = nlp(u"Mary slapped the green witch.")
for chunk in doc.noun_chunks:
    print('{} - {}'.format(chunk, chunk.label_))

Mary - NP
the green witch - NP


## 7. 문장 구조
- 구문 분석parsing : 구 단위를 식별하는 부분 구문 분석과 달리 구 사이의 관계를 파악하는 작업
- 구분 분석 트리parse tree는 문장안의 문법 요소가 계층적으로 어떻게 관련되는지를 보여준다
    - 구성 구문 분석 constituent parsing
    - 의존 구문 분석 dependency parsing

## 8. 단어 의미와 의미론
- 단어에는 의미가 하나 이상 있고 단어가 나타내는 각각의 뜻을 단어의 의미sense라고 한다
- 단어 의미는 문맥으로 결정될 수 있다
- 텍스트에서 단어 의미를 자동으로 찾는 일은 실제로 NLP에 적용된 첫 번째 준지도 학습 semi-supervised learning 이다